In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
from adjustText import adjust_text
from pdb import set_trace as st 
import random
import json
import os 
from tqdm import tqdm

In [9]:
def format_length_with_sqrt(length, tolerance=1e-10):
    """
    Format length as square root if it's the square root of an integer.
    Returns a tuple: (formatted_string, is_sqrt_form)
    """
    # Check if the length squared is close to an integer
    length_squared = length ** 2
    nearest_int = round(length_squared)
    
    if abs(length_squared - nearest_int) < tolerance and nearest_int > 0:
        # Check if the length itself is not already an integer
        if abs(length - round(length)) > tolerance:
            return f'√{nearest_int}', True
        else:
            # Length is already an integer, return as is
            return str(int(round(length))), False
    else:
        # Not a perfect square root, return rounded decimal
        return str(round(length, 2)), False

def draw_shortest_arc(start_point, center, end_point, num_points=100):
    """Draw an arc from start_point to end_point around center"""
    start_vec = start_point - center
    end_vec = end_point - center
    
    start_angle = np.arctan2(start_vec[1], start_vec[0])
    end_angle = np.arctan2(end_vec[1], end_vec[0])
    
    radius = np.linalg.norm(start_vec)
    
    # Ensure we take the shorter arc
    angle_diff = end_angle - start_angle
    if angle_diff > np.pi:
        angle_diff -= 2 * np.pi
    elif angle_diff < -np.pi:
        angle_diff += 2 * np.pi
    
    angles = np.linspace(start_angle, start_angle + angle_diff, num_points)
    arc_x = center[0] + radius * np.cos(angles)
    arc_y = center[1] + radius * np.sin(angles)
    
    plt.plot(arc_x, arc_y, 'black', linewidth=1.5)

def smart_label_offset(x, y, vertices, offset_distance=0.2):
    """Calculate smart offset for vertex labels to avoid overlaps"""
    # Convert vertices to array for easier calculation
    coords = np.array([[v["x"], v["y"]] for v in vertices.values()])
    current_point = np.array([x, y])
    
    # Find the direction that has the most space
    directions = np.array([[1, 1], [-1, 1], [1, -1], [-1, -1], 
                          [1, 0], [-1, 0], [0, 1], [0, -1]])
    
    best_direction = directions[0]
    max_min_distance = 0
    
    for direction in directions:
        test_point = current_point + direction * offset_distance
        # Calculate minimum distance to all other points
        distances = np.linalg.norm(coords - test_point, axis=1)
        distances = distances[distances > 0]  # Remove distance to itself
        
        if len(distances) > 0:
            min_distance = np.min(distances)
            if min_distance > max_min_distance:
                max_min_distance = min_distance
                best_direction = direction
    
    return current_point + best_direction * offset_distance
def draw_mark(xs,ys):
    vector = [np.diff(xs)[0],np.diff(ys)[0]]
    vector = np.array([vector[1],-vector[0]])
    vector_size = np.sqrt((vector**2).sum())
    vector *= 0.5/vector_size
    plt.plot([xs.mean()-vector[0],xs.mean()+vector[0]],[ys.mean()-vector[1],ys.mean()+vector[1]],color = 'black')
def plot_geometry_improved(description):
    labels = []
    
    # Set up the plot with some padding
    all_x = [v["x"] for v in description["vertices"].values()]
    all_y = [v["y"] for v in description["vertices"].values()]
    x_range = max(all_x) - min(all_x)
    y_range = max(all_y) - min(all_y)
    padding = max(x_range, y_range) * 0.1
    
    ax.set_xlim(min(all_x) - padding, max(all_x) + padding)
    ax.set_ylim(min(all_y) - padding, max(all_y) + padding)
    
    # Plot vertices with smart label positioning
    for v in description["vertices"]:
        point = description["vertices"][v]
        #ax.scatter(point["x"], point["y"], color='black', s=50, zorder=5)
        
        # Use smart offset for vertex labels
        offset_pos = smart_label_offset(point["x"], point["y"], 
                                      description["vertices"], offset_distance=0.4)
        
        label = ax.text(offset_pos[0], offset_pos[1], v, fontsize=14, 
                       fontweight='bold', ha='center', va='center',
                       bbox=dict(boxstyle="round,pad=0.2", facecolor='white', 
                                edgecolor='none', alpha=0.8))
        labels.append(label)
    
    # Plot segments
    segment_labels = []
    for s in description["segments"]:
        start, end = s
        xs = np.array([description["vertices"][start]["x"], 
                      description["vertices"][end]["x"]])
        ys = np.array([description["vertices"][start]["y"], 
                      description["vertices"][end]["y"]])
        if s == f"{vertex_perm[1]}D" or s == f"D{vertex_perm[2]}":
            draw_mark(xs,ys)
        ax.plot(xs, ys, color='black', linewidth=2, zorder=1)
        if description["segments"][s]["known"] or s == description["question"]["target"]:
            length = np.sqrt(np.diff(xs)[0]**2 + np.diff(ys)[0]**2)
            if description["segments"][s]["known"]:
                description["segments"][s]["length"] = np.around(length,2)
                description["segments"][s]["unit"] = ["cm"]
            # Format the length with square root notation if applicable
            formatted_length, is_sqrt = format_length_with_sqrt(length)
            
            # Calculate perpendicular offset for segment labels
            segment_vector = np.array([np.diff(xs)[0], np.diff(ys)[0]])
            segment_length = np.linalg.norm(segment_vector)
            perpendicular = np.array([-segment_vector[1], segment_vector[0]]) / segment_length
            
            # Offset the label perpendicular to the segment
            offset_distance = 0.2
            label_x = xs.mean() + perpendicular[0] * offset_distance
            label_y = ys.mean() + perpendicular[1] * offset_distance
            
            # Choose background color based on whether it's a square root
            bg_color = 'lightyellow' if is_sqrt else 'lightblue'
            if s == description["question"]["target"]:
                label = ax.text(label_x, label_y, f'{s}=?', fontsize=12,
                           ha='center', va='center',
                           bbox=dict(boxstyle="round,pad=0.2", facecolor=bg_color, 
                                    edgecolor='none', alpha=0.8))

            else:
                label = ax.text(label_x, label_y, f'{s}={formatted_length}', fontsize=12,
                           ha='center', va='center',
                           bbox=dict(boxstyle="round,pad=0.2", facecolor=bg_color, 
                                    edgecolor='none', alpha=0.8))
            segment_labels.append(label)
    
    # Plot angles
    angle_labels = []
    for a in description["angles"]:
        start, vertex, end = a
        if not description["angles"][a]["known"]:
            continue
        
        a_point = np.array([description["vertices"][start]["x"], 
                           description["vertices"][start]["y"]], dtype=np.float64)
        b_point = np.array([description["vertices"][vertex]["x"], 
                           description["vertices"][vertex]["y"]], dtype=np.float64)
        c_point = np.array([description["vertices"][end]["x"], 
                           description["vertices"][end]["y"]], dtype=np.float64)
        
        vec_a = a_point - b_point 
        vec_c = c_point - b_point
        a_size = np.sqrt((vec_a**2).sum())
        c_size = np.sqrt((vec_c**2).sum())
        degree = np.around(np.arccos((vec_a*vec_c).sum()/(a_size*c_size))*180/np.pi,2)
        description["angles"][a]["value"] = degree
        description["angles"][a]["unit"] = "deg"
        radius = 0.2 * min(a_size, c_size)
        
        vec_a /= a_size
        vec_c /= c_size
        
        if degree == 90:
            # Draw right angle marker
            corner_size = radius * 0.7
            corner1 = b_point + vec_a * corner_size
            corner2 = b_point + vec_c * corner_size
            corner3 = corner1 + vec_c * corner_size
            
            ax.plot([corner1[0], corner3[0]], [corner1[1], corner3[1]], 
                   'black', linewidth=2)
            ax.plot([corner2[0], corner3[0]], [corner2[1], corner3[1]], 
                   'black', linewidth=2)
        else:
            # Draw arc for other angles
            draw_shortest_arc(b_point + vec_a * radius, b_point, 
                            b_point + vec_c * radius, num_points=50)
            
            # Position angle label
            vec_mid = vec_a + vec_c
            vec_mid_normalized = vec_mid / np.sqrt((vec_mid**2).sum())
            label_radius = radius * 1.3  # Place label outside the arc
            
            label_pos = b_point + vec_mid_normalized * label_radius
            label = ax.text(label_pos[0], label_pos[1], f'{degree}°', fontsize=12,
                           ha='center', va='center', color='black', fontweight='bold',
                           bbox=dict(boxstyle="round,pad=0.2", facecolor='lightgreen', 
                                    edgecolor='none', alpha=0.8))
            angle_labels.append(label)
    
    # Combine all labels for adjustment
    all_labels = labels + segment_labels + angle_labels
    
    # Get all non-text artists (points, lines) to avoid
    avoid_objects = [artist for artist in ax.get_children() 
                    if not isinstance(artist, plt.Text)]
    
    # Fine-tune the label positioning
    adjust_text(all_labels, ax=ax,
                add_objects=avoid_objects,
                expand_points=(1.5, 1.5),    # Distance from points
                expand_text=(1.2, 1.2),     # Distance between labels
                expand_objects=(1.2, 1.2),  # Distance from lines/objects
                arrowprops=dict(arrowstyle='-', color='gray', alpha=0.5, lw=0.5),
                force_points=(0.5, 0.5),    # Force to avoid points
                force_text=(0.5, 0.5),      # Force to avoid text overlap
                force_objects=(0.3, 0.3),   # Force to avoid other objects
                lim=1000)                   # Maximum iterations
    
    ax.set_aspect('equal')
    ax.axis('off')
#    ax.set_title('Geometry Problem', 
 #fontsize=16, fontweight='bold', pad=20)
    
    plt.tight_layout()
    return description


def convert_numpy_types(obj):
    if isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, (np.int_, np.intc, np.intp, np.int8, np.int16, np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_numpy_types(v) for v in obj]
    else:
        return obj



In [13]:
# Your original description
def save_json(description, output_dir, idx):
#    os.makedirs(f"{output_dir}/labels", exist_ok=True)
    with open(f"{output_dir}/labels/{idx:06d}.json", 'w') as f:
        json.dump(description, f, indent=2)  # indent=2 for human readability
        
np.random.seed(42)#np.random.randint(100))  # For reproducible results
output_dir = "/home/yfrid/Desktop/stem-whiteboard/dataset"

ABC = ["A","B","C"]

for idx in tqdm(range(1000,2000)):
    fig, ax = plt.subplots(figsize=(8,8),dpi = 32)
    ### Randomize Values
    Height = np.random.choice(np.arange(3,10))
    Width = np.random.choice(np.arange(3,10))
    rotation =np.random.choice(np.arange(0,360,5))
    mirror = bool(random.getrandbits(1))
    if mirror:
        Width *=-1
    theta = rotation*np.pi/180

    ### Vertex permutation
    vertex_perm = np.random.permutation(ABC)

    segment_names = [f"{vertex_perm[0]}{vertex_perm[1]}",f"{vertex_perm[0]}{vertex_perm[2]}",
                     f"{vertex_perm[0]}D",f"{vertex_perm[1]}{vertex_perm[2]}",
                     f"{vertex_perm[1]}D",f"D{vertex_perm[2]}"]
    ### Segments mask - which values are given, what is the target
    perm = np.random.permutation(range(4))
    segments = perm[:1]
    if segments.max() == 3:
        segments[segments.argmax()] = np.random.choice(range(3,6))
    segments_mask = np.zeros(6,dtype = bool)
    segments_mask[segments] = True
    target = segment_names[perm[2]]
    if target == 3:
        target = np.random.choice(range(3,6))
    segments_mask
    description = {
        "vertices": {
            f"{vertex_perm[0]}": {"x": 0, "y": 0},
            f"{vertex_perm[1]}": {"x": -np.sin(theta)*Height, "y": np.cos(theta)*Height},
            f"{vertex_perm[2]}": {"x": np.cos(theta)*Width, "y": np.sin(theta)*Width},
#            f"{vertex_perm[2]}": {"x": np.cos(theta)*Width-np.sin(theta)*Height, "y": np.cos(theta)*Height+np.sin(theta)*Width},
            "D": {"x": np.cos(theta)*Width/2-np.sin(theta)*Height/2, "y": np.cos(theta)*Height/2+np.sin(theta)*Width/2}
        },
        "segments": {
            segment_names[0]: {"known": segments_mask[0]},
            segment_names[1]: {"known": segments_mask[1]},
            segment_names[2]: {"known": segments_mask[2]},
            segment_names[3]: {"known": segments_mask[3]},
            segment_names[4]: {"known": segments_mask[4]},
            segment_names[5]: {"known": segments_mask[5]},
        },
        "angles": {
            f"{vertex_perm[1]}{vertex_perm[0]}{vertex_perm[2]}": {"known": True},
            f"{vertex_perm[0]}{vertex_perm[1]}{vertex_perm[2]}": {"known": True},
            f"{vertex_perm[1]}{vertex_perm[2]}{vertex_perm[0]}": {"known": False},
            f"{vertex_perm[1]}D{vertex_perm[0]}": {"known": False},
            f"{vertex_perm[1]}{vertex_perm[0]}D": {"known": False}
        },
        "specials":{"medians":[{"from":vertex_perm[0],"to":"D","of_side":f"{vertex_perm[1]}{vertex_perm[2]}"}]
        },
        "question": {
            "type": "find_length",
            "target": f"{target}",
            "question_text": f"Find the length of {target}"
        }
    }
    new = plot_geometry_improved(description)
    xlims = np.array(ax.get_xlim())
    
    ylims = np.array(ax.get_ylim())
    xs = []
    ys = []
    for v in new["vertices"]:
        xs.append(new["vertices"][v]["x"])
        ys.append(new["vertices"][v]["y"])


    xlims[0] = min(xlims[0],np.min(np.array(xs))-1)
    xlims[1] = max(xlims[1],np.max(np.array(xs))+1)
    
    ylims[0] = min(ylims[0],np.min(np.array(ys))-1)
    ylims[1] = max(ylims[1],np.max(np.array(ys))+1)
    
        
    for v in new["vertices"]:
        new["vertices"][v]["x"] = np.around((new["vertices"][v]["x"]-xlims[0])/(xlims[1]-xlims[0]),3)
        new["vertices"][v]["y"] = np.around((new["vertices"][v]["y"]-ylims[0])/(ylims[1]-ylims[0]),3)
    plt.savefig(f"{output_dir}/images/{idx:06d}.png", dpi=48, facecolor='white')
    save_json(convert_numpy_types(new), output_dir, idx)
    
    plt.close()

100%|███████████████████████████████████████| 1000/1000 [02:45<00:00,  6.05it/s]
